In [1]:
# Imports 
import numpy as np
import random
from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [90, 4],
 'London': [55, 47],
 'Moscow': [62, 43],
 'Barcelona': [98, 3],
 'Rome': [6, 27],
 'Paris': [18, 31],
 'Vienna': [36, 64],
 'Munich': [95, 96],
 'Istanbul': [12, 23],
 'Kyiv': [52, 36],
 'Bucharest': [10, 57],
 'Minsk': [63, 38],
 'Warsaw': [50, 80],
 'Budapest': [88, 96],
 'Milan': [81, 32],
 'Prague': [24, 70],
 'Sofia': [38, 61],
 'Birmingham': [13, 40],
 'Brussels': [90, 9],
 'Amsterdam': [86, 38]}

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Munich', 'Istanbul', 'Moscow', ..., 'Barcelona', 'Milan',
        'Bucharest'],
       ['Kyiv', 'Budapest', 'Minsk', ..., 'Milan', 'Vienna', 'Barcelona'],
       ['London', 'Moscow', 'Vienna', ..., 'Amsterdam', 'Barcelona',
        'Bucharest'],
       ...,
       ['Prague', 'Moscow', 'Istanbul', ..., 'Munich', 'Rome', 'Vienna'],
       ['Milan', 'Barcelona', 'Amsterdam', ..., 'Birmingham', 'Paris',
        'Sofia'],
       ['Bucharest', 'Kyiv', 'Milan', ..., 'Warsaw', 'Paris', 'Moscow']],
      dtype='<U10')

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1086.43644092, 1074.13072816,  936.24528477,  929.2581587 ,
        911.75366381,  920.60319826, 1093.05866673, 1008.45555162,
       1094.95663099, 1065.89956628, 1017.05296908, 1059.177982  ,
        866.45592175, 1029.27748758, 1159.37589981,  954.3877005 ,
        927.58648908,  995.62946164, 1008.0333238 ,  907.49439634,
        965.22797455, 1022.74913773,  999.4128244 , 1100.04400715,
        947.84536368,  957.25076158, 1039.51277513,  921.63511031,
        969.70545183,  926.26249846, 1069.3643686 , 1005.57359736,
       1070.13466977, 1007.0187767 ,  939.62385461,  952.58789661,
       1013.98561736,  770.55276183,  899.80763526, 1033.91748002,
       1020.49533123,  949.50287045, 1085.40436842, 1058.59579148,
        941.16442409,  812.29940553,  972.02071808,  876.58043066,
       1016.379481  , 1118.61900887,  968.15071529, 1106.64662842,
       1107.56852803, 1068.77426162, 1040.61992738,  901.78023691,
       1095.2634749 ,  967.65319343, 1036.02650712, 1038.65934

In [7]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['London', 'Moscow', 'Vienna', 'Kyiv', 'Paris', 'Warsaw', 'Minsk',
       'Rome', 'Istanbul', 'Birmingham', 'Berlin', 'Budapest', 'Brussels',
       'Munich', 'Prague', 'Sofia', 'Milan', 'Amsterdam', 'Barcelona',
       'Bucharest'], dtype='<U10')

In [8]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Vienna', 'Rome', 'Barcelona', 'Sofia', 'Bucharest', 'Munich',
       'Istanbul', 'Moscow', 'London', 'Warsaw', 'Birmingham', 'Prague',
       'Berlin', 'Minsk', 'Budapest', 'Amsterdam', 'Paris', 'Brussels',
       'Kyiv', 'Milan'], dtype='<U10')

In [9]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Vienna', 'Kyiv', 'Barcelona', 'Bucharest', 'Sofia', 'Moscow',
       'Milan', 'London', 'Munich', 'Warsaw', 'Birmingham', 'Prague',
       'Budapest', 'Minsk', 'Berlin', 'Amsterdam', 'Paris', 'Brussels',
       'Rome', 'Istanbul'], dtype='<U10')

In [10]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%50==0: print(i, best_solution[1], fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 inf 998.5047178501486 25/09/22 16:23
50 604.4560782199642 1003.187627349624 25/09/22 16:23
100 604.4560782199642 1008.8082746365465 25/09/22 16:23
150 585.7172946410959 1023.3656214201262 25/09/22 16:23
200 585.7172946410959 1013.2637805376379 25/09/22 16:23
250 585.7172946410959 1021.6112175287769 25/09/22 16:23
300 585.7172946410959 1002.0081056105234 25/09/22 16:24
350 585.7172946410959 1004.8992176424207 25/09/22 16:24
400 529.242685960957 1002.6900079874661 25/09/22 16:24
450 529.242685960957 1021.1726833665916 25/09/22 16:24
500 529.242685960957 996.2582232028091 25/09/22 16:24
550 529.242685960957 1016.5940830274164 25/09/22 16:24
600 529.242685960957 1009.5217256093109 25/09/22 16:24
650 529.242685960957 1010.8096248267093 25/09/22 16:24
700 529.242685960957 1000.8753494098361 25/09/22 16:24
750 529.242685960957 1016.3851405284388 25/09/22 16:24
800 529.242685960957 1015.0722516551164 25/09/22 16:24
850 529.242685960957 1010.9162242851709 25/09/22 16:24
900 529.242685960957 9

In [11]:
best_solution

[385,
 529.242685960957,
 array([['Amsterdam', 'Warsaw', 'Prague', 'Vienna', 'Moscow', 'Milan',
         'Brussels', 'Barcelona', 'Berlin', 'Kyiv', 'London', 'Sofia',
         'Munich', 'Budapest', 'Minsk', 'Birmingham', 'Bucharest',
         'Paris', 'Rome', 'Istanbul']], dtype='<U10')]